NAME: YASH PATNI

ROLL NO.: A4-71

PRACTICAL NO. 3

AIM:

(A) Write a program to find FIRST for any grammar. All the following rules of FIRST must be implemented.

(B) Calculate Follow for the given grammar and Construct the LL (1) parsing table using the FIRST and FOLLOW.

In [1]:
import sys
from tabulate import tabulate

terminals = ['a', 'b', 'c', 'p']
non_terminals = ['S', 'A', 'B', 'C']
starting_symbol = 'S'
productions = ["S->ABC/C", "A->a/bB/e", "B->p/e", "C->c"]
productions_dict = {nT: [] for nT in non_terminals}

for production in productions:
    nonterm_to_prod = production.split("->")
    alternatives = nonterm_to_prod[1].split("/")
    for alternative in alternatives:
        productions_dict[nonterm_to_prod[0]].append(alternative)

LL1_table = {}
conflicts = False

def first(string):
    first_ = set()
    if string in non_terminals:
        alternatives = productions_dict[string]
        for alternative in alternatives:
            first_2 = first(alternative)
            first_ |= first_2
    elif string in terminals:
        first_ = {string}
    elif string == '' or string == 'e':
        first_ = {'e'}
    else:
        first_2 = first(string[0])
        if 'e' in first_2:
            i = 1
            while 'e' in first_2:
                first_ |= (first_2 - {'e'})
                if string[i:] in terminals:
                    first_ |= {string[i:]}
                    break
                elif string[i:] == '':
                    first_ |= {'e'}
                    break
                first_2 = first(string[i:])
                first_ |= (first_2 - {'e'})
                i += 1
        else:
            first_ |= first_2
    return first_

def follow(nT):
    follow_ = set()
    prods = productions_dict.items()
    if nT == starting_symbol:
        follow_ |= {'$'}
    for nt, rhs in prods:
        for alt in rhs:
            for char in alt:
                if char == nT:
                    following_str = alt[alt.index(char) + 1:]
                    if following_str == '':
                        if nt == nT:
                            continue
                        else:
                            follow_ |= follow(nt)
                    else:
                        follow_2 = first(following_str)
                        if 'e' in follow_2:
                            follow_ |= (follow_2 - {'e'})
                            follow_ |= follow(nt)
                        else:
                            follow_ |= follow_2
    return follow_

FIRST = {non_terminal: set() for non_terminal in non_terminals}
FOLLOW = {non_terminal: set() for non_terminal in non_terminals}

for non_terminal in non_terminals:
    FIRST[non_terminal] |= first(non_terminal)

FOLLOW[starting_symbol] |= {'$'}
for non_terminal in non_terminals:
    FOLLOW[non_terminal] |= follow(non_terminal)

print("{: ^20}{: ^20}{: ^20}".format('Non Terminals', 'First', 'Follow'))
for non_terminal in non_terminals:
    print("{: ^20}{: ^20}{: ^20}".format(non_terminal, str(FIRST[non_terminal]), str(FOLLOW[non_terminal])))

for non_terminal, alternatives in productions_dict.items():
    for alternative in alternatives:
        first_set_alt = first(alternative)
        for terminal in first_set_alt - {'e'}:
            if (non_terminal, terminal) not in LL1_table:
                LL1_table[(non_terminal, terminal)] = alternative
            else:
                print(f"Conflict at ({non_terminal}, {terminal})")
                conflicts = True

        if 'e' in first_set_alt or '' in first_set_alt:  # If epsilon is in FIRST(α)
            for terminal in FOLLOW[non_terminal]:
                if (non_terminal, terminal) not in LL1_table:
                    LL1_table[(non_terminal, terminal)] = alternative
                else:
                    print(f"Conflict at ({non_terminal}, {terminal})")
                    conflicts = True

# Displaying the LL(1) parsing table
table_data = []
for key, value in LL1_table.items():
    table_data.append([key[0], key[1], value])

print(tabulate(table_data, headers=['Non Terminal', 'Terminal', 'Production'], tablefmt='grid'))

if not conflicts:
    print("No conflicts found.")

   Non Terminals           First               Follow       
         S          {'b', 'a', 'p', 'c'}       {'$'}        
         A            {'b', 'a', 'e'}        {'p', 'c'}     
         B               {'p', 'e'}          {'p', 'c'}     
         C                 {'c'}               {'$'}        
Conflict at (S, c)
Conflict at (B, p)
+----------------+------------+--------------+
| Non Terminal   | Terminal   | Production   |
+================+============+==============+
| S              | b          | ABC          |
+----------------+------------+--------------+
| S              | a          | ABC          |
+----------------+------------+--------------+
| S              | p          | ABC          |
+----------------+------------+--------------+
| S              | c          | ABC          |
+----------------+------------+--------------+
| A              | a          | a            |
+----------------+------------+--------------+
| A              | b          | bB           |